# QM9 Molecule Dataset [1, 2]

#### Format of the XYZ-like files [1]

- 1$^{st}$ line: number of atoms ($n_a$) that composes the molecule


- 2$^{nd}$ line: scalar properties, as defined by the following sequence

| index number | observable | units | description |
| :- | :- | :- | :- |
| 1 | tag | | gdb9 string |
| 2 | identifier | | Unique integer identifier of molecule |
| 3 | A | GHz |	Rotational constant |
| 4 | B | GHz |	Rotational constant |
| 5 | C | GHz |	Rotational constant |
| 6 | μ | Debye | Dipole moment magnitude |
| 7 | α | $a^3_0$ | Isotropic polarizability |
| 8 | ϵHOMO | Hartree | Energy of HOMO |
| 9 | ϵLUMO | Hartree | Energy of LUMO |
| 10 | ϵgap | Hartree |Gap (ϵLUMO−ϵHOMO) |
| 11 | 〈R2〉 | $a^2_0$ | Electronic spatial extent |
| 12 | zpve | Hartree | Zero point vibrational energy |
| 13 | U(0K) | Hartree | Internal energy at 0.0 K |
| 14 | U(298K) | Hartree | Internal energy at 298.15 K |
| 15 | H(298K) | Hartree | Enthalpy at 298.15 K |
| 16 | G(298K) | Hartree | Free energy at 298.15 K |
| 17 | Cv(298K) | $\frac{cal}{mol K}$ | Heat capacity at 298.15 K |

- 3$^{rd}$ to ($n_a$ + 2) lines: properties as defined by the following sequence

| description | units |
| :- | :- |
| Element abbreviation (e.g. C) |   |
| Coordinate x | Å | 
| Coordinate y | Å |
| Coordinate z | Å |
| Mulliken partial atomic charges | e |

- $n_a$ + 3 lines: harmonic vibrational frequencies (cm$^{-1}$), where the total number of frequence are defined by
    - linear molecules = $3 * n_a − 5$
    - non-linear molecules $3 * n_a − 6$
    
    
- $n_a$ + 4 lines: SMILES strings from GDB-17 and from B3LYP relaxation


- $n_a$ + 5 lines: InChI strings for Corina and B3LYP geometries

---
#### Sources:
1. Ramakrishnan, R., Dral, P., Rupp, M. et al. Quantum chemistry structures and properties of 134 kilo molecules. Sci Data 1, 140022 (2014). https://doi.org/10.1038/sdata.2014.22
2. http://quantum-machine.org/datasets

In [1]:
import csv
import sys

from pathlib import Path

Specify the following directories:
1. the path to the original qm9 XYZ-like files (i.e. they do have .xyz file extensions, even though the are not XYZ files)
2. the path to where you want to save the cleaned up files

---

#### Note
Below for testing, I have 10 qm9 XYZ-like files in a subdirectory called `structures_tests`
 (e.g. cp structures/dsgdb9nsd_00001*.xyz structures_tests/).
 
I am saving the cleaned up files to `structures_clean`.

For the full downloaded data set, I untarred the file (`dsgdb9nsd.xyz.tar`) in the folder `structures`. Know that there are a large number of files that results.

---

In [2]:
path_qm9_original = Path('structures_tests')
path_qm9_clean = Path('structures_clean')

In [3]:
def replace_hidden_char(input_file: str=None, source_dir: str=None, output_dir: str=None):
    ''' Replaces the following hidden characters in a file:
            a. tabs (i.e. `\t`)
            b. double while spaces (i.e. `  `), and
            c. end-of-line while spaces, while preserving the carriage return

        The improved new file is save in a specified directory.

        Input
            input_file: file name
            source_dir: directory for where the input_file is located
            output_dir: directory for where the ouput_file will be written
        Output
            a new file with the same prefix name, but whose extension is changed
                to 'txt' and then saved to the output_dir
    '''

    if not isinstance(input_file, str):
        sys.exit(f'Error: {input_file} is not a string.')
    elif not isinstance(source_dir, str):
        sys.exit(f'Error: {source_dir} is not a string.')
    elif not isinstance(output_dir, str):
        sys.exit(f'Error: {output_dir} is not a string.')
    else:
        output = input_file.replace(source_dir, output_dir).replace('xyz', 'txt')

        with open(input_file) as file_in, open(output, 'w') as file_out:
            for line in file_in:
                file_out.write(line.replace('\t', ' ').replace('  ', ' ').replace(' \n', '\n'))

In [4]:
files = list(path_qm9_original.glob('*.xyz'))

for file in files:
    replace_hidden_char(input_file=str(file),
                        source_dir=str(path_qm9_original),
                        output_dir=str(path_qm9_clean))

In [5]:
def process_qm9_file(input_file: str=None, source_dir: str=None):
    ''' Process a single XYZ-like file from the QM9 dataset.

        Input
            input_file: file name
            source_dir: directory for where the input_file is located
        Return
            n_atoms (int): number of atoms in molecule
            scalar_dict (dictionary): scalar properties
            coords (list): element, x coordinate, y coordiante, z coordinate, Mulliken partial atomic charge
            freq (list): vibrational frequencies
            smiles (list): SMILES strings
            inchi (list): InChi strings
    '''

    n_atoms = None
    scalar_values = None
    coords = []
    frequencies = None
    smiles = None
    inchi = None

    if not isinstance(input_file, str):
        sys.exit(f'Error: {input_file} is not a string.')
    elif not isinstance(source_dir, str):
        sys.exit(f'Error: {source_dir} is not a string.')
    else:
        with open(input_file) as file:
            for line_number, line in enumerate(file):
                if line_number == 0:
                    n_atoms = int(line)
                elif line_number == 1:
                    scalar_values = line.split()
                elif 2 <= line_number <= n_atoms + 1:   ## coordinates
                    for atom in range(0, n_atoms, 1):
                        if line_number == atom + 2:     ## skip the first two lines
                            coords.append(line.split())
                elif line_number == n_atoms + 2:
                    freq = line.split()
                elif line_number == n_atoms + 3:
                    smiles = line.split()
                elif line_number == n_atoms + 4:
                    inchi = line.split()

    if len(coords) != n_atoms:
        sys.exit(f'Error: the n_atoms does not match the number of coordinate sets (x,y,z) '
                 f'read in ({input_file}).')

    scalar_observables = ['tag', 'number',
                          'A', 'B', 'C',
                          'mu', 'alpha',
                          'HOMO', 'LUPO', 'gap',
                          'r2',
                          'ZPVE', 'U0', 'U', 'H', 'G', 'Cv']

    if len(scalar_values) == 17:
        scalar_dict = dict(zip(scalar_observables, scalar_values))
    else:
        sys.exit(f'Error: there is an incorrect number of scalar propertes ({input_file} line 2).')

    return n_atoms, scalar_dict, coords, freq, smiles, inchi

In [6]:
def split_coord_mulliken(elem_coord_mulliken: list=None, input_file: str=None):
    ''' Takes the 'coordinate' part of QM9's XYZ-like file and
            splits it into true XYZ content (i.e. element, x,y,z-coordinates)
            and the Mulliken partial atomic charges (PAC).
            
        Input
            elem_coord_mulliken (list of lists): element, x,y,z-coordinates, Mulliken PAC
        Output
            a new file that represents a true XYZ formatted file, including the
                QM9 molecule idenifier
        Return
            atom_xyz (list of lists): element, x,y,z-coordinates
            mulliken_dict (dict): a dictionary that provides the element and mulliken charge
    '''
    
    if not isinstance(elem_coord_mulliken, list):
        sys.exit(f'Error: {elem_coord_mulliken} is not a list.')
    elif not isinstance(input_file, str):
        sys.exit(f'Error: {input_file} is not a string.')
    else:
        ## create and save a true xyz formatted fiel
        atom_xyz = [elem_xyz[:-1] for elem_xyz in elem_coord_mulliken]

        output = input_file.replace('txt', 'xyz')
        
        with open(output,"w") as file:
            writer = csv.writer(file, delimiter=' ', quoting = csv.QUOTE_NONE)
            writer.writerows([{n_atoms}])
            writer.writerows([[f"{scalar_dict['tag']}_molecule_number_{scalar_dict['number']}"]])
            writer.writerows(atom_xyz)

        ## create mulliken partial atomic charge dictionary
        element_number = []
        mulliken_pac = []
        
        for number, content in enumerate(elem_coord_mulliken):
            element_number.append(f'{content[0]}{number}')
            mulliken_pac.append(float(content[-1]))

        mulliken_dict = dict(zip(element_number, mulliken_pac))
        
        return atom_xyz, mulliken_dict

In [7]:
files = list(path_qm9_clean.glob('*.txt'))

for file in files:
    print(str(file))
    process_qm9_file(input_file=str(file), source_dir=str(path_qm9_clean))
    n_atoms, scalar_dict, coords, freq, smiles, inchi = process_qm9_file(input_file=str(file),
                                                                         source_dir=str(path_qm9_clean))

    atom_xyz, mulliken_charges = split_coord_mulliken(elem_coord_mulliken=coords, input_file=str(file))

    print(n_atoms)
    print(scalar_dict)
    print(atom_xyz)
    print(freq)
    print(smiles)
    print(inchi)
    print(mulliken_charges)

    linear = None
    if 3 * n_atoms - 5 == len(freq):
        linear = True
    elif 3 * n_atoms - 6 == len(freq):
        linear = False
    print(f'Molecule has a linear geometry: {linear}')

    print()

structures_clean/dsgdb9nsd_000012.txt
6
{'tag': 'gdb', 'number': '12', 'A': '73.8472', 'B': '11.34793', 'C': '9.83639', 'mu': '3.7286', 'alpha': '21.57', 'HOMO': '-0.2543', 'LUPO': '0.0302', 'gap': '0.2845', 'r2': '145.3078', 'ZPVE': '0.045279', 'U0': '-169.860788', 'U': '-169.856903', 'H': '-169.855958', 'G': '-169.885594', 'Cv': '10.89'}
[['N', '-0.0258998583', '1.34614561', '0.0088939089'], ['C', '0.0464672751', '-0.0117434132', '0.0012042484'], ['O', '1.0718352049', '-0.6525878165', '-0.0111331282'], ['H', '0.8253549651', '1.8850494841', '0.0037376912'], ['H', '-0.9083767959', '1.8267963784', '0.018919919'], ['H', '-0.9614410008', '-0.4750037629', '0.0080739107']]
['144.7399', '563.1718', '647.4334', '1052.0167', '1054.3274', '1270.9954', '1431.0761', '1614.5936', '1834.1837', '2928.8538', '3596.5407', '3737.3526']
['NC=O', 'NC=O']
['InChI=1S/CH3NO/c2-1-3/h1H,(H2,2,3)', 'InChI=1S/CH3NO/c2-1-3/h1H,(H2,2,3)']
{'N0': -0.463064, 'C1': 0.169707, 'O2': -0.304836, 'H3': 0.264237, 'H4': 0.